# 2. Modélisation avec Vertex AI Forecast - Chicago Taxi Demand

Ce notebook couvre l'entraînement d'un modèle de prévision de la demande de taxis à Chicago en utilisant Vertex AI Forecast. Nous allons :
- Initialiser l'environnement Vertex AI
- Charger la configuration depuis le fichier YAML
- Créer un dataset de séries temporelles à partir des données BigQuery
- Configurer et lancer un job d'entraînement AutoML Forecasting
- Analyser les résultats et les métriques d'évaluation

## 1. Configuration et Initialisation

Importons les bibliothèques nécessaires et initialisons l'environnement.

In [2]:
# Bibliothèques standards
import os
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Google Cloud & Vertex AI
from google.cloud import aiplatform
from google.cloud import bigquery

# Configuration visuelle
%matplotlib inline
sns.set(style="whitegrid", context="talk")
plt.rcParams['figure.figsize'] = [12, 8]

## 2. Configuration du Projet GCP

Définissons les identifiants du projet et initialisons Vertex AI.

In [3]:
# Configuration du projet GCP
# Load from YAML first
try:
    with open("../config/pipeline_config.yaml", "r") as f:
        config = yaml.safe_load(f)
    print("Configuration chargée avec succès depuis pipeline_config.yaml")
    PROJECT_ID = config.get('gcp', {}).get('project_id', 'avisia-certification-ml-yde') 
    REGION = config.get('gcp', {}).get('region', 'europe-west1')
    BQ_DATASET = config.get('bigquery', {}).get('dataset_id', 'chicago_taxis')  
    BUCKET_URI = config.get('gcp', {}).get('bucket_uri', f"gs://{PROJECT_ID}-vertex-bucket")  
    BQ_SOURCE_URI = f"bq://{PROJECT_ID}.{BQ_DATASET}.demand_by_hour"
    # Add defaults if keys might be missing
except Exception as e:
    print(f"Impossible de charger le fichier de configuration: {e}. Using default values.")
    config = {}
    PROJECT_ID = "avisia-certification-ml-yde" # Fallback
    REGION = "europe-west1" # Fallback
    BQ_DATASET = "chicago_taxis" # Fallback

print(f"Using PROJECT_ID: {PROJECT_ID}, REGION: {REGION}, BQ_DATASET: {BQ_DATASET}")
# Initialisation de Vertex AI
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

print(f"✅ Vertex AI initialisé avec succès pour le projet {PROJECT_ID}")
# Initialisation du client BigQuery
try:
    client = bigquery.Client(project=PROJECT_ID, location=REGION) # Spécifier location peut aider
    print(f"✅ Client BigQuery initialisé pour le projet {PROJECT_ID} en région {REGION}")
except Exception as e:
    print(f"Erreur lors de l'initialisation du client BigQuery: {e}")
    # Gérer l'erreur ou arrêter si le client est indispensable
    client = None # Ou raise SystemExit("Client BigQuery requis")

Configuration chargée avec succès depuis pipeline_config.yaml
Using PROJECT_ID: avisia-certification-ml-yde, REGION: europe-west1, BQ_DATASET: chicago_taxis
✅ Vertex AI initialisé avec succès pour le projet avisia-certification-ml-yde


/Users/gustavevernay/Desktop/Projets/Pro/Avisia/chicago-taxi-genai-demo/venv/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


✅ Client BigQuery initialisé pour le projet avisia-certification-ml-yde en région europe-west1


## 3. Chargement de la Configuration

Chargeons les paramètres de configuration depuis le fichier YAML.

In [5]:
# Chargement du fichier de configuration
try:
    with open("../config/pipeline_config.yaml", "r") as f:
        config = yaml.safe_load(f)
    print("✅ Configuration chargée avec succès.")
except FileNotFoundError:
    try:
        with open("config/pipeline_config.yaml", "r") as f:
            config = yaml.safe_load(f)
        print("✅ Configuration chargée avec succès.")
    except FileNotFoundError:
        print("⚠️ Fichier de configuration introuvable. Utilisation des valeurs par défaut.")
        config = {
            "forecasting": {
                "time_column": "timestamp_hour",
                "target_column": "trip_count",
                "context_column": "pickup_community_area",
                "forecast_horizon": 24,
                "window_size": 168,
                "available_at_forecast": [
                    "timestamp_hour", "day_of_year", "day_of_week", "hour", 
                    "month", "is_weekend"
                ],
                "unavailable_at_forecast": ["trip_count"],
                "data_granularity_unit": "hour"
            },
            "vertex_ai_forecast": {
                "display_name": "chicago_taxi_forecast_model",
                "optimization_objective": "minimize-rmse",
                "budget_milli_node_hours": 100
            }
        }

# Extraction des paramètres de configuration
forecast_config = config["forecasting"]
vertex_config = config["vertex_ai_forecast"]
time_window = forecast_config["time_window"]
max_data_points = time_window["max_data_points"]
if max_data_points is None:
    max_data_points = 2950
end_date = time_window["end_date"] 
end_date = datetime.strptime(end_date, "%Y-%m-%d")
start_date = end_date - timedelta(hours=max_data_points)
# Paramètres clés pour le forecasting
time_column = forecast_config["time_column"]
target_column = forecast_config["target_column"]
context_column = forecast_config["context_column"]
forecast_horizon = forecast_config["forecast_horizon"]
window_size = forecast_config["window_size"]
available_at_forecast = forecast_config["available_at_forecast"]
unavailable_at_forecast = forecast_config["unavailable_at_forecast"]

# Affichage des principaux paramètres
print(f"\nParamètres de forecasting:")
print(f"- Colonne temporelle: {time_column}")
print(f"- Colonne cible: {target_column}")
print(f"- Identifiant de série: {context_column}")
print(f"- Horizon de prévision: {forecast_horizon} heures")
print(f"- Taille de la fenêtre historique: {window_size} heures")

✅ Configuration chargée avec succès.

Paramètres de forecasting:
- Colonne temporelle: timestamp_hour
- Colonne cible: trip_count
- Identifiant de série: pickup_community_area
- Horizon de prévision: 24 heures
- Taille de la fenêtre historique: 168 heures


## 4. Création du Dataset de Séries Temporelles

 Créons un dataset de séries temporelles dans Vertex AI à partir de la table BigQuery
 `demand_by_hour` préparée et filtrée par le Notebook 1.


In [ ]:
# Nom d'affichage du dataset
dataset_display_name = f"{vertex_config['display_name']}-dataset-{datetime.now().strftime('%Y%m%d_%H%M')}"

# Vérifier que BQ_SOURCE_URI est défini (devrait venir du début du notebook ou du config)
if 'BQ_SOURCE_URI' not in locals():
    # Recalculer BQ_SOURCE_URI à partir de PROJECT_ID et BQ_DATASET si nécessaire
    BQ_SOURCE_URI = f"bq://{PROJECT_ID}.{BQ_DATASET}.demand_by_hour"
    print(f"BQ_SOURCE_URI défini à : {BQ_SOURCE_URI}")
print(f"Utilisation de la source BigQuery: {BQ_SOURCE_URI}")


# Création du dataset avec la table filtrée
try:
    # Utiliser la table BigQuery filtrée au lieu de la table entière
    dataset = aiplatform.TimeSeriesDataset.create(
        display_name=dataset_display_name,
        bq_source=BQ_SOURCE_URI,  # Utiliser la table filtrée
    )
    print(f"✅ Dataset créé avec données filtrées: {dataset.resource_name}")
    print(f"Lien vers le dataset dans la console Cloud:")
    print(f"https://console.cloud.google.com/vertex-ai/locations/{REGION}/datasets/{dataset.name}?project={PROJECT_ID}")

except Exception as e:
    print(f"⚠️ Erreur lors de la création du dataset: {e}")
    print("Tentative de récupération d'un dataset existant...")
    try:
        # Rechercher parmi les datasets existants
        datasets = aiplatform.TimeSeriesDataset.list(
            filter=f'display_name="{dataset_display_name}"',
            order_by="create_time desc",
            location=REGION,
            project=PROJECT_ID
        )
        if datasets:
            dataset = datasets[0]
            print(f"✅ Dataset existant récupéré: {dataset.resource_name}")
            print(f"Lien: https://console.cloud.google.com/vertex-ai/locations/{REGION}/datasets/{dataset.name}?project={PROJECT_ID}")
            # Ajouter une vérification des données si possible/nécessaire
            print(f"   Source BQ associée: {getattr(dataset, 'metadata', {}).get('inputConfig', {}).get('bigquerySource', {}).get('uri', 'Non trouvée')}")
            print(f"⚠️ ATTENTION: Ce dataset pourrait contenir plus de 3000 points par série!")
        else:
            print(f"❌ Aucun dataset existant trouvé avec le nom : {dataset_display_name}")
            dataset = None # Assurer que dataset est None si échec

    except Exception as e2:
        print(f"❌ Erreur lors de la récupération du dataset: {e2}")
# Vérifier si le dataset a bien été créé ou récupéré
if not dataset:
    print("❌ Impossible de continuer sans dataset Vertex AI.")
    # Vous pourriez vouloir arrêter l'exécution ici: 
    raise SystemExit("Erreur critique: Dataset non disponible")

/Users/gustavevernay/Desktop/Projets/Pro/Avisia/chicago-taxi-genai-demo/venv/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Creating TimeSeriesDataset
Create TimeSeriesDataset backing LRO: projects/807699310940/locations/europe-west1/datasets/4187740723036028928/operations/3513644162819817472
Create TimeSeriesDataset backing LRO: projects/807699310940/locations/europe-west1/datasets/4187740723036028928/operations/3513644162819817472
TimeSeriesDataset created. Resource name: projects/807699310940/locations/europe-west1/datasets/4187740723036028928
To use this TimeSeriesDataset in another session:
ds = aiplatform.TimeSeriesDataset('projects/807699310940/locations/europe-west1/datasets/4187740723036028928')
✅ Dataset créé avec données filtrées: projects/807699310940/locations/europe-west1/datasets/4187740723036028928


## 5. Configuration et Lancement du Job d'Entraînement

Configurons et lançons un job d'entraînement AutoML Forecasting.

In [8]:
# Nom du job d'entraînement
job_display_name = f"taxi_demand_forecast_job_{datetime.now().strftime('%Y%m%d_%H%M')}"

# Configuration des transformations de colonnes
# Définir explicitement les transformations pour chaque colonne sauf pickup_community_area
formatted_transformations = [
    {"timestamp": {"column_name": time_column}},  # La colonne timestamp
    {"numeric": {"column_name": target_column}},   # La colonne cible
    
    # Features supplémentaires disponibles à l'heure de prévision
    {"numeric": {"column_name": "day_of_year"}},
    {"numeric": {"column_name": "day_of_week"}},
    {"numeric": {"column_name": "hour"}},
    {"numeric": {"column_name": "month"}},
    {"categorical": {"column_name": "is_weekend"}}
]



print(f"Transformations de colonnes configurées: {len(formatted_transformations)} colonnes")
print(f"Note: {context_column} (identifiant de série) est EXCLU des transformations pour éviter les conflits")

Transformations de colonnes configurées: 7 colonnes
Note: pickup_community_area (identifiant de série) est EXCLU des transformations pour éviter les conflits


In [9]:
# Création du job d'entraînement
training_job = aiplatform.AutoMLForecastingTrainingJob(
    display_name=job_display_name,
    optimization_objective=vertex_config.get("optimization_objective", "minimize-rmse"),
    column_transformations=formatted_transformations,
)

print(f"✅ Job d'entraînement configuré: {job_display_name}")

✅ Job d'entraînement configuré: taxi_demand_forecast_job_20250415_1103


## Dataset Splitting Justification

Pour respecter la temporalité (éviter les fuites), nous avons fait :
- 80% des dates les plus anciennes pour le train
- 10% (période intermédiaire) pour la validation
- 10% (période la plus récente) pour le test

Cela reflète un scénario réaliste où l'on prévoit sur des dates futures.

In [92]:
# Nom du modèle
model_display_name = f"{vertex_config['display_name']}_{datetime.now().strftime('%Y%m%d_%H%M')}"

# S'assurer que context_column n'est PAS dans available_at_forecast ni unavailable_at_forecast
available_at_forecast_cleaned = [col for col in available_at_forecast if col != context_column]
unavailable_at_forecast_cleaned = [col for col in unavailable_at_forecast if col != context_column]

# Informer l'utilisateur des modifications
if context_column in available_at_forecast or context_column in unavailable_at_forecast:
    print(f"ℹ️ Note: {context_column} a été retiré des listes de colonnes car c'est l'identifiant de série temporelle.")
    print(f"L'identifiant de série ne doit pas apparaître dans les listes des colonnes disponibles ou indisponibles.")

# Lancement de l'entraînement
print(f"⏳ Démarrage de l'entraînement du modèle... Cela peut prendre plusieurs heures.")
try:
    model = training_job.run(
        dataset=dataset,
        target_column=target_column,
        time_column=time_column,
        time_series_identifier_column=context_column,
        # Utiliser les listes nettoyées sans l'identifiant de série
        unavailable_at_forecast_columns=unavailable_at_forecast_cleaned,
        available_at_forecast_columns=available_at_forecast_cleaned,
        # L'identifiant de série n'a pas besoin d'être spécifié comme attribut de série
        time_series_attribute_columns=[],
        forecast_horizon=forecast_horizon,
        context_window=window_size,
        data_granularity_unit=forecast_config.get("data_granularity_unit", "hour"),
        data_granularity_count=1,
        budget_milli_node_hours=vertex_config.get("budget_milli_node_hours", 100),
        model_display_name=model_display_name,
        training_fraction_split=vertex_config.get("training_fraction_split", 0.8),
        validation_fraction_split=vertex_config.get("validation_fraction_split", 0.1),
        test_fraction_split=vertex_config.get("test_fraction_split", 0.1),
        export_evaluated_data_items=True,
        sync=True,  # Mode synchrone: attend la fin de l'entraînement
    )
    print(f"✅ Modèle entraîné avec succès: {model.display_name}")
    print(f"Resource name: {model.resource_name}")
except Exception as e:
    print(f"❌ Erreur lors de l'entraînement du modèle: {e}")
    print("\nDétails complets de l'erreur pour analyse:")
    import traceback
    print(traceback.format_exc())

⏳ Démarrage de l'entraînement du modèle... Cela peut prendre plusieurs heures.
View Training:
https://console.cloud.google.com/ai/platform/locations/europe-west1/training/3734390157390905344?project=807699310940
AutoMLForecastingTrainingJob projects/807699310940/locations/europe-west1/trainingPipelines/3734390157390905344 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLForecastingTrainingJob projects/807699310940/locations/europe-west1/trainingPipelines/3734390157390905344 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLForecastingTrainingJob projects/807699310940/locations/europe-west1/trainingPipelines/3734390157390905344 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLForecastingTrainingJob projects/807699310940/locations/europe-west1/trainingPipelines/3734390157390905344 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLForecastingTrainingJob projects/807699310940/locations/europe-west1/trainingPipelines/3734390157390905344 current state:
Pipel

## 6. Évaluation du Modèle

Examinons les métriques d'évaluation du modèle entraîné.

In [ ]:
try:
    # Récupération des métriques d'évaluation
    evaluation = model.get_model_evaluation()
    metrics = evaluation.metrics
    
    print("Métriques d'évaluation :")
    for metric_name, metric_value in metrics.items():
        if isinstance(metric_value, (int, float)):
            print(f"- {metric_name}: {metric_value:.4f}")
        else:
            print(f"- {metric_name}: {metric_value}")
except Exception as e:
    print(f"⚠️ Impossible de récupérer les métriques d'évaluation: {e}")
    print("Les métriques peuvent ne pas être disponibles immédiatement après l'entraînement.")

⚠️ Impossible de récupérer les métriques d'évaluation: name 'model' is not defined
Les métriques peuvent ne pas être disponibles immédiatement après l'entraînement.


## 7. Importance des Features

Analysons l'importance des features dans le modèle.

In [ ]:
try:
    # Récupération de l'importance des features
    feature_importance = model.get_feature_importance()
    
    if feature_importance:
        # Conversion en DataFrame pour faciliter la visualisation
        feature_importance_df = pd.DataFrame({
            'Feature': [item.feature_id for item in feature_importance],
            'Importance': [item.importance_score for item in feature_importance]
        })
        
        # Tri par importance décroissante
        feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)
        
        # Affichage
        print("Importance des features :")
        print(feature_importance_df)
        
        # Visualisation
        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
        plt.title('Importance des Features', fontsize=16)
        plt.xlabel('Importance', fontsize=14)
        plt.ylabel('Feature', fontsize=14)
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ Aucune information d'importance des features disponible.")
except Exception as e:
    print(f"⚠️ Impossible de récupérer l'importance des features: {e}")

⚠️ Impossible de récupérer l'importance des features: name 'model' is not defined


## 8. Sauvegarde des Informations du Modèle

Sauvegardons les informations du modèle pour une utilisation ultérieure.

In [ ]:
# Sauvegarde des informations du modèle dans un fichier pour référence ultérieure
try:
    model_info = {
        'model_name': model.display_name,
        'resource_name': model.resource_name,
        'create_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'project': PROJECT_ID,
        'region': REGION,
        'forecast_horizon': forecast_horizon,
        'window_size': window_size,
    }
    
    # Création du répertoire si nécessaire
    os.makedirs('outputs', exist_ok=True)
    
    # Sauvegarde dans un fichier YAML
    with open(f'outputs/model_info_{datetime.now().strftime("%Y%m%d_%H%M")}.yaml', 'w') as f:
        yaml.dump(model_info, f)
    
    print(f"✅ Informations du modèle sauvegardées.")
except Exception as e:
    print(f"⚠️ Impossible de sauvegarder les informations du modèle: {e}")

⚠️ Impossible de sauvegarder les informations du modèle: name 'model' is not defined


## 10. Informations de Dépannage

### Erreurs courantes et solutions

1. **Erreur**: `Found time series with more than 3000 time steps`
   - **Solution**: Nous avons ajouté une logique pour limiter la longueur des séries temporelles en ajustant la plage de dates. Si vous rencontrez toujours cette erreur, réduisez davantage la plage de dates.

2. **Erreur**: `The columns 'pickup_community_area' have transformation but do not have any time dependency properties`
   - **Solution**: Nous avons exclu l'identifiant de série (pickup_community_area) des transformations et des listes de colonnes.

3. **Performances du modèle faibles**:
   - **Solution**: Vous pouvez augmenter `budget_milli_node_hours` pour permettre à AutoML d'explorer plus de modèles.
   - Essayez d'ajouter plus de variables contextuelles ou météorologiques.
   - Ajustez l'horizon de prévision et la taille de la fenêtre de contexte.

## 9. Conclusion et Prochaines Étapes

### Résumé
- Nous avons créé un dataset de séries temporelles à partir des données BigQuery
- Nous avons configuré et entraîné un modèle de forecasting avec Vertex AI AutoML
- Nous avons évalué les performances du modèle et analysé l'importance des features

### Prochaines Étapes
- Utiliser le modèle pour générer des prédictions batch sur des périodes futures
- Visualiser et analyser les résultats de prévision
- Explorer d'autres configurations de modèle pour améliorer les performances

Le notebook suivant (3_Batch_Prediction_Visualization.ipynb) abordera la génération et la visualisation des prédictions.